# Préparation des données de départ

### def `get_first_data`

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from time import sleep


def get_first_data(nb):
    """
    20 éléments par page
    ~1 minute par page
    """

    links = []
    nb_cards = 0
    data = {}

    driver = Chrome()
    driver.get('https://howlongtobeat.com/?q=')
    sleep(2)
    driver.find_element(By.ID, 'onetrust-accept-btn-handler').click()

    print(f'Récupération des {nb} premiers liens en cours...')

    while nb_cards < nb:
        sleep(2)
        cards = driver.find_elements(By.CLASS_NAME, 'GameCard_inside_blur__cP8_l')
        for card in cards:
            link = card.find_element(By.TAG_NAME, 'a').get_attribute('href')

            try:
                links.index(link)
                continue
            except:
                links.append(link)
                pass
            nb_cards = nb_cards + 1
            if nb_cards >= nb:
                break
            pass
        driver.find_element(By.CLASS_NAME, 'Pagination_right__GwBE_.form_button.Pagination_inactive__dnoZF').click()
        pass

    print(f'Récupération des {nb} premiers liens terminés')

    print('Récupération des données en cours...')

    for link in links:
        driver.get(link)
        sleep(2)

        try: id = driver.current_url.split('/')[-1]
        except: id = 'none'

        try: title = driver.find_element(By.CLASS_NAME, 'GameHeader_profile_header__q_PID.shadow_text').text
        except: title = 'none'

        try: img = driver.find_element(By.CLASS_NAME, 'GameSideBar_game_image__ozUTt.mobile_hide').find_element(By.TAG_NAME, 'img').get_attribute('src').split('?')[0]
        except: img = 'none'

        try: link = driver.current_url
        except: link = 'none'

        try: duration_main = driver.find_elements(By.CLASS_NAME, 'GameStats_short__tSJ6I.time_100')[0].find_element(By.TAG_NAME, 'h5').text
        except: duration_main = 'none'

        try: duration_extra = driver.find_elements(By.CLASS_NAME, 'GameStats_short__tSJ6I.time_100')[1].find_element(By.TAG_NAME, 'h5').text
        except: duration_extra = 'none'

        try: duration_complete = driver.find_elements(By.CLASS_NAME, 'GameStats_short__tSJ6I.time_100')[2].find_element(By.TAG_NAME, 'h5').text
        except: duration_complete = 'none'

        try: driver.find_element(By.ID, 'profile_summary_more').click()
        except: pass

        try: desc = driver.find_elements(By.CLASS_NAME, 'GameSummary_profile_info__HZFQu')[0].text
        except: desc = 'none'

        try: platform = driver.find_elements(By.CLASS_NAME, 'GameSummary_profile_info__HZFQu')[2].text.split('\n')[1]
        except: platform = 'none'

        try: genre = driver.find_elements(By.CLASS_NAME, 'GameSummary_profile_info__HZFQu')[3].text.split('\n')[1]
        except: genre = 'none'

        try: developer = driver.find_elements(By.CLASS_NAME, 'GameSummary_profile_info__HZFQu')[4].text.split('\n')[1]
        except: developer = 'none'

        try: publisher = driver.find_elements(By.CLASS_NAME, 'GameSummary_profile_info__HZFQu')[5].text.split('\n')[1]
        except: publisher = 'none'

        data[id] = {
            'title': title,
            'img': img,
            'link': link,
            'duration_main': duration_main,
            'duration_extra': duration_extra,
            'duration_complete': duration_complete,
            'platform': platform,
            'genre': genre,
            'developer': developer,
            'publisher': publisher,
            'desc': desc
        }
        pass

    print('Récupération des données terminées')

    driver.close()
    return data

### Récupération des premières données

In [2]:
data = get_first_data(20)

Récupération des 20 premiers liens en cours...
Récupération des 20 premiers liens terminés
Récupération des données en cours...
Récupération des données terminées


# Création de la base de données

### class `Database`

In [3]:
import sqlalchemy as db


class DataBase():
    def __init__(self, name_database='database'):
        self.name = name_database
        self.url = f"sqlite:///{name_database}.db"
        self.engine = db.create_engine(self.url)
        self.connection = self.engine.connect()
        self.metadata = db.MetaData()
        self.table = self.engine.table_names()

    def create_table(self, name_table, **kwargs):
        colums = [db.Column(k, v, primary_key = True) if 'id_' in k else db.Column(k, v) for k,v in kwargs.items()]
        db.Table(name_table, self.metadata, *colums)
        self.metadata.create_all(self.engine)
        print(f"Table : '{name_table}' are created succesfully")

    def read_table(self, name_table, return_keys=False):
        table = db.Table(name_table, self.metadata, autoload=True, autoload_with=self.engine)
        if return_keys:table.columns.keys()
        else : return table

    def add_row(self, name_table, **kwarrgs):
        table = self.read_table(name_table)

        stmt = (db.insert(table).values(kwarrgs))
        self.connection.execute(stmt)
        print(f'Row added')

    def delete_row_by_id(self, name_table, id_):
        table = self.read_table(name_table)

        stmt = (db.delete(table).where(table.columns.id_ == id_))
        self.connection.execute(stmt)
        print(f'Row id {id_} deleted')

    def select_table(self, name_table):
        table = self.read_table(name_table)
        stmt = db.select([table])
        return self.connection.execute(stmt).fetchall()

    def delete_table(self, name_table):
        return self.connection.execute(f'DROP TABLE {name_table}')

### Initialisation de la base de données

In [4]:
database = DataBase('database')

C:\Users\jerem\AppData\Local\Temp\ipykernel_16544\1173263507.py:11: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  self.table = self.engine.table_names()


### Création de la table `games`

In [5]:
database.create_table(
    "games",
    id_=db.Integer,
    title=db.String,
    img=db.String,
    link=db.String,
    duration_main=db.String,
    duration_extra=db.String,
    duration_complete=db.String,
    platform=db.String,
    genre=db.String,
    developer=db.String,
    publisher=db.String,
    desc=db.String,
)

Table : 'games' are created succesfully


### Insertion des données dans `games`

In [6]:
for i in data:
    database.add_row(
        "games",
        title=data[i]["title"],
        img=data[i]["img"],
        link=data[i]["link"],
        duration_main=data[i]["duration_main"],
        duration_extra=data[i]["duration_extra"],
        duration_complete=data[i]["duration_complete"],
        platform=data[i]["platform"],
        genre=data[i]["genre"],
        developer=data[i]["developer"],
        publisher=data[i]["publisher"],
        desc=data[i]["desc"],
    )

Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added
Row added


In [22]:
title = 'titslsqde'
img = 'imgsqd'
link = 'linkqsd'
duration_main = 'qsdduration_main'
duration_extra = 'duration_extra'
duration_complete = 'duration_complete'
platform = 'platform'
genre = 'genre'
developer = 'developer'
publisher = 'publisher'
desc = 'desc'

In [23]:
title

'titslsqde'

In [25]:
data = [title, img, link]

In [26]:
(title, img, link) = data